In [ ]:
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

url = "https://www.ycombinator.com/companies"

# Set up options for the Chrome WebDriver
options = webdriver.ChromeOptions()
# options.add_argument("--headless") # Run the browser in headless mode (without GUI)

# Initialize the WebDriver with the specified options
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(
    5
)  # Set an implicit wait time of 5 seconds for elements to be found

# Open the URL in the browser
driver.get(url)
time.sleep(20)  # Wait for 20 seconds to ensure the page loads

# Set the number of pages to scroll and the delay between each scroll
pages = 2
page_delay = 2

# Loop to scroll down the page multiple times to load more content dynamically
for i in range(pages):  # Scroll down x times
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(page_delay)
    print(f"Page {i + 1} of {pages} loaded")

# Get the page source and close the driver
page_source = driver.page_source
driver.quit()

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")
box = soup.find("div", class_="_rightCol_1pgsr_592") #  _rightCol_86jzd_575
#<<Phung start note
#print(box)
if box is None:
    print("Không tìm thấy phần tử chứa dữ liệu!")
    driver.quit()
    exit()
#>>
companies = [["Title", "Address", "Description", "Batch", "Industry", "Tags"]]

# Find all company elements
company_elements = box.find_all("a", class_="_company_1pgsr_355") #_company_86jzd_338

for element in company_elements:
    title = element.find("span", class_="_coName_1pgsr_470").get_text(strip=True)  #_coName_86jzd_453

    address = element.find("span", class_="_coLocation_1pgsr_486").get_text(strip=True) #_coLocation_86jzd_469

    description = element.find("span", class_="_coDescription_1pgsr_495").get_text(   #_coDescription_86jzd_478
        strip=True
    )

    spans = element.find_all("a", class_="_tagLink_1pgsr_1040") #_tagLink_86jzd_1023

    batch = spans[0].get_text(strip=True)

    industry = spans[1].get_text(strip=True)

    if len(spans) >= 3:
        tag = spans[2].get_text(strip=True)
    else:
        tag = "N/A"

    companies.append([title, address, description, batch, industry, tag])

# Convert to DataFrame
df = pd.DataFrame(companies[1:], columns=companies[0])

Page 1 of 2 loaded
Page 2 of 2 loaded
<div class="_rightCol_1pgsr_592"><div class="_section_1pgsr_163 _searchBox_1pgsr_202"><input class="" placeholder="Search..." type="text" value=""/></div><div class="_showFilters_1pgsr_166"><button>Filter</button></div><div class="_message_1pgsr_559">Showing 80 of 1,000+ companies</div><div class="_section_1pgsr_163 _results_1pgsr_343"><a class="_company_1pgsr_355" href="/companies/airbnb"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img alt="" class="rounded-full bg-gray-100" role="presentation" src="https://bookface-images.s3.amazonaws.com/small_logos/3e9a0092bee2ccf926e650e59c06503ec6b9ee65.png"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_1pgsr_470">Airbnb</span><span class="_coLocation_1pgsr_486">San Francisco, CA, USA</span></div><div><span class="_coDescription_1pgsr_495">Book accommodations

In [ ]:
df

,Title,Address,Description,Batch,Industry,Tags
0,Airbnb,"San Francisco, CA, USA",Book accommodations around the world.,W09,Consumer,"Travel, Leisure and Tourism"
1,Amplitude,"San Francisco, CA, USA",Digital Analytics Platform,W12,B2B,Analytics
2,Coinbase,"San Francisco, CA, USA","Buy, sell, and manage cryptocurrencies.",S12,Fintech,Banking and Exchange
3,DoorDash,"San Francisco, CA, USA",Restaurant delivery.,S13,Consumer,Food and Beverage
4,Dropbox,"San Francisco, CA, USA",Backup and share files in the cloud.,S07,B2B,Productivity
...,...,...,...,...,...,...
75,Nowports,,The core engine of supply chain in LATAM,W19,B2B,Supply Chain and Logistics
76,Odeko,"New York, NY, USA",Our operations software makes it easier to run...,S19,B2B,N/A
77,GitPrime,"Durango, CO, USA",Visibility and analytics for software developm...,W16,B2B,"Engineering, Product and Design"
78,Proxy,"San Francisco, CA, USA",Smart ring digital wallet,S16,Consumer,Consumer Electronics


In [11]:
df.to_csv("Ycombinator.csv", index = False)